<a href="https://colab.research.google.com/github/Jxiang2/CMPUT366/blob/main/comput366_assign3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
%tensorflow_version 1.x

In [33]:

"""
Utility functions for cnn.py
"""
import tensorflow as tf
import numpy as np

def get_data(name):
    mnist = tf.keras.datasets.mnist
    (x_train, y_train),(x_test, y_test) = mnist.load_data()
    x_train = x_train / 255.0
    x_test = x_test / 255.0
    
    with tf.Session():
         trn_y = tf.one_hot(y_train, 10).eval()
         tst_y = tf.one_hot(y_test, 10).eval()

    if name == 'top_left':
        trn_x = decenter(x_train, -2)
        tst_x = decenter(x_test, -2)
    elif name == 'bottom_right':
        trn_x = decenter(x_train, 2)
        tst_x = decenter(x_test, 2)
    else:
        raise ValueError("Only valid names are ['top_left', 'bottom_right'], not '%s'" % name)

    return (trn_x,trn_y), (tst_x,tst_y)

def decenter(X,pad=2):
    out = np.zeros([X.shape[0], X.shape[1]+abs(pad), X.shape[2]+abs(pad)])
    if pad > 0:
        out[:,pad:X.shape[1]+pad, pad:X.shape[2]+pad] = X
    else:
        out[:,0:X.shape[1], 0:X.shape[2]] = X
    return out

def show_examples(X1, Y1, X2, Y2, fname='examples.png'):
    import matplotlib
    matplotlib.use('Agg')
    import matplotlib.pyplot as plt
    plt.figure(figsize=(1,5))
    for digit in range(10):
        i = Y1[:,digit].argmax()
        plt.subplot(10, 2, 2*digit+1)
        frame1 = plt.gca()
        frame1.axes.get_xaxis().set_visible(False)
        frame1.axes.get_yaxis().set_visible(False)
        
        plt.imshow(X1[i], cmap='gray')

        plt.subplot(10, 2, 2*digit+2)
        frame1 = plt.gca()
        frame1.axes.get_xaxis().set_visible(False)
        frame1.axes.get_yaxis().set_visible(False)

        plt.imshow(X2[i], cmap='gray')
    plt.savefig(fname)

In [34]:
"""
Solution stub for Question 2 (Neural Networks).

Fill in the implementations of the `mlp2` and `cnn` functions.

See https://www.tensorflow.org/tutorials for a Tensorflow tutorial.
"""
from __future__ import print_function
import numpy as np
import tensorflow as tf

# These should be the only tensorflow classes you need:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D

# get_data returns (train_x, train_y), (test_x, test_y)
# argument determines whether images are shifted to top-left or bottom-right
# X values are an array of 30x30 images
# Y values are an array of 10 one-hot encoded labels
#from cnn_utils import get_data(need to be recovered)

# show_examples creates an image that shows some example data from two datasets
# side by side
#from cnn_utils import show_examples(need to be recovered)


def mlp1(train_x, train_y, test1_x, test1_y, test2_x, test2_y):
    """
    Train and evaluate a feedforward network with a single hidden layer.
    """
    model = Sequential([
      Flatten(input_shape=(30, 30)), # Need to flatten before Dense layers
      Dense(512, activation='relu'),
      Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(train_x, train_y, epochs=5)

    print("Evaluating MLP1 on test set 1")
    model.evaluate(test1_x, test1_y)
    print("Evaluating MLP1 on test set 2")
    return model.evaluate(test2_x, test2_y)

def mlp2(train_x, train_y, test1_x, test1_y, test2_x, test2_y):
    """
    Train and evaluate a feedforward network with two hidden layers.
    """
    # First layer will need argument `input_shape=(30,30)`
    model = Sequential([
      # TODO: add your implementation here
      Flatten(input_shape=(30, 30)),
      Dense(128, activation='relu'),
      Dense(64, activation='relu'),
      Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(train_x, train_y, epochs=5)

    print("Evaluating MLP2 on test set 1")
    model.evaluate(test1_x, test1_y)
    print("Evaluating MLP2 on test set 2")
    return model.evaluate(test2_x, test2_y)


def cnn(train_x, train_y, test1_x, test1_y, test2_x, test2_y):
    """
    Train and evaluate a feedforward network with two hidden layers.
    """
    # Add a single "channels" dimension at the end
    trn_x = train_x.reshape([-1, 30, 30, 1])
    tst1_x = test1_x.reshape([-1, 30, 30, 1])
    tst2_x = test2_x.reshape([-1, 30, 30, 1])

    # First layer will need argument `input_shape=(30,30,1)`
    model = Sequential([
        # TODO: add your implementation here
        Conv2D(32, (5,5), strides = (1,1), input_shape=(30,30,1)),
        MaxPooling2D((2,2), strides = (2,2)),
        Conv2D(64, (5,5)),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(trn_x, train_y, epochs=5)

    print("Evaluating CNN on test set 1")
    model.evaluate(tst1_x, test1_y)
    print("Evaluating CNN on test set 2")
    return model.evaluate(tst2_x, test2_y)

def main():
    (train1_x, train1_y), (test1_x, test1_y) = get_data('top_left')
    (train2_x, train2_y), (test2_x, test2_y) = get_data('bottom_right')

    # Left column is images from top_left dataset
    # Right column is corresponding images from bottom_right dataset
    show_examples(test1_x, test1_y, test2_x, test2_y, 'examples.png')

    mlp1(train1_x, train1_y, test1_x, test1_y, test2_x, test2_y)
    mlp2(train1_x, train1_y, test1_x, test1_y, test2_x, test2_y)
    cnn(train1_x, train1_y, test1_x, test1_y, test2_x, test2_y)


if __name__ == '__main__':
    main()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 13s 211us/sample - loss: 0.2025 - acc: 0.9396
Epoch 2/5
60000/60000 [==============================] - 12s 205us/sample - loss: 0.0812 - acc: 0.9750
Epoch 3/5
60000/60000 [==============================] - 12s 206us/sample - loss: 0.0510 - acc: 0.9845
Epoch 4/5
60000/60000 [==============================] - 13s 209us/sample - loss: 0.0370 - acc: 0.9880
Epoch 5/5
60000/60000 [==============================] - 12s 206us/sample - loss: 0.0270 - acc: 0.9916
Evaluating MLP1 on test set 1
10000/10000 [==============================] - 1s 140us/sample - loss: 0.0687 - acc: 0.9805
Evaluating MLP1 on test set 2
10000/10000 [==============================] - 1s 111us/sample - loss: 2.2740 - acc: 0.5847
Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 9s 146us/sample - loss: 0.2427 - acc: 0.9293
Epoch 2/5
60000/60000 [==============================] - 8s 142us/sample - loss: 0.1031 - acc